In [1]:
# Importing Pandas an Numpy Libraries to use on manipulating our Data
import pandas as pd
import numpy as np

# To Preproccesing our data
from sklearn.preprocessing import LabelEncoder

# To fill missing values
from sklearn.impute import SimpleImputer

# To Split our train data
from sklearn.model_selection import train_test_split

# To Visualize Data
import matplotlib.pyplot as plt
import seaborn as sns

# To Train our data
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, GaussianNB

# To evaluate end result we have 
from sklearn.metrics import accuracy_score, confusion_matrix,mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

In [ ]:
#We are importing our Data with Pandas Library
# We use "Coronary_artery.csv" 
df = pd.read_csv("../Data/AdSmartABdata.csv")
df.head()

In [ ]:
clean_data = df[df['yes'] == 1].append(df[df['no'] == 1]) 
 

In [ ]:
clean_data.isna().sum()

In [ ]:
clean_data['date'] = pd.to_datetime(df['date'], errors='coerce')

clean_data['date_of_week'] = clean_data['date'].dt.day_name()

In [ ]:
clean_data.drop(['no'], axis=1,inplace=True)
clean_data.rename(columns={'yes': 'result'},inplace=True)

In [ ]:
exposed_group = clean_data[clean_data['experiment'] == 'exposed']
control_group = clean_data[clean_data['experiment'] == 'control']